1.- ***Instalación y configuración.***

TTS es compatible con Python 3.7 a <3.11.0 y ha sido probado en Ubuntu 18.10, 19.10 y 20.10.


*Instalación desde PyPI*

In [ ]:
# Instalar TTS desde PyPI
!pip install TTS


*Instalación desde Github*

In [ ]:
# Instalar TTS desde GitHub
!pip install git+https://github.com/coqui-ai/TTS


*Instalación desde sistemas Linux. *

In [ ]:
# Clonar el repositorio desde GitHub
!git clone https://github.com/coqui-ai/TTS/

# Navegar al directorio del repositorio
%cd TTS

# Instalar las dependencias del sistema (solo en Linux)
!make system-deps

# Instalar TTS
!make install




---



***2.-Configuración y Ejecución del Entrenamiento del Modelo***



1. Importaciones y Configuración de Rutas:

2. Definir Configuración del Dataset:
3. Inicializar el Procesador de Audio:
4. Inicializar el Tokenizador:
5. Cargar Muestras de Datos:
6. Inicializar el Modelo:
7. Inicializar el Entrenador:
8. Entrenamiento





In [ ]:
import os

# Importar módulos necesarios
from trainer import Trainer, TrainerArgs
from TTS.tts.configs.glow_tts_config import GlowTTSConfig
from TTS.tts.configs.shared_configs import BaseDatasetConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.models.glow_tts import GlowTTS
from TTS.tts.utils.text.tokenizer import TTSTokenizer
from TTS.utils.audio import AudioProcessor

# Definir la ruta de salida como la misma que el script
output_path = os.path.dirname(os.path.abspath(__file__))

# Definir la configuración del dataset
dataset_config = BaseDatasetConfig(
    formatter="ljspeech", meta_file_train="metadata.csv", path=os.path.join(output_path, "../LJSpeech-1.1/")
)

# Inicializar el procesador de audio
ap = AudioProcessor.init_from_config(config)

# Inicializar el tokenizador
tokenizer, config = TTSTokenizer.init_from_config(config)

# Cargar muestras de datos para entrenamiento y evaluación
train_samples, eval_samples = load_tts_samples(
    dataset_config,
    eval_split=True,
    eval_split_max_size=config.eval_split_max_size,
    eval_split_size=config.eval_split_size,
)

# Inicializar la configuración del modelo
config = GlowTTSConfig(
    batch_size=32,
    eval_batch_size=16,
    num_loader_workers=4,
    num_eval_loader_workers=4,
    run_eval=True,
    test_delay_epochs=-1,
    epochs=1000,
    text_cleaner="phoneme_cleaners",
    use_phonemes=True,
    phoneme_language="en-us",
    phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
    print_step=25,
    print_eval=False,
    mixed_precision=True,
    output_path=output_path,
    datasets=[dataset_config],
)

# Inicializar el modelo
model = GlowTTS(config, ap, tokenizer, speaker_manager=None)

# Inicializar el entrenador
trainer = Trainer(
    TrainerArgs(), config, output_path, model=model, train_samples=train_samples, eval_samples=eval_samples
)

# Iniciar el entrenamiento
trainer.fit()


***3.Parámetros de Preprocesamiento y Ejemplo de Audio***

Aspectos a considerar.

  **Tiempo:** ¿Cuál es la duración del audio que se considera adecuada? ¿Hay alguna restricción en la duración?

  **Frecuencia:** ¿Cuál es la frecuencia de muestreo necesaria para el modelo? ¿Cómo asegurarse de que todos los audios tengan esta frecuencia?
    
  **Transformaciones:** ¿Qué transformaciones son necesarias para normalizar el audio? ¿Se necesita una transformación lineal o alguna otra técnica de ajuste?
  
  **Longitud de los Segmentos de Audio:**
  Algunos modelos requieren que los audios se dividan en segmentos de longitud fija. Especifica si se debe cortar o rellenar el audio para que se ajuste a esta longitud.


**Ventanas y Superposición:**
  Si se utilizan técnicas como la ventana de Fourier para la transformación del audio, menciona el tamaño de la ventana y el grado de superposición.

**Especificaciones de Formato:**

  Algunas veces el modelo puede tener requisitos específicos sobre el formato del archivo (por ejemplo, WAV, FLAC) o sobre la profundidad de bits.

**Normalización del Volumen:**

  Describe cómo ajustar el volumen del audio para asegurar que todos los archivos tengan un nivel de audio consistente.

**Ruido de Fondo y Limpieza:**

  Si el modelo requiere que el audio esté libre de ruido de fondo, menciona cualquier proceso de limpieza que deba aplicarse.

**Padding y Trimming:**

  Explica cómo se deben manejar los audios que son demasiado cortos (relleno) o demasiado largos (recorte).

**Especificaciones del Texto:**

  Si el modelo también necesita que el texto esté en un formato específico (por ejemplo, sin puntuación, en minúsculas), incluye detalles sobre cómo preparar y formatear el texto.

3.2 Preprocesamiento de Audio en Python

In [ ]:
import os
import numpy as np
import librosa
from TTS.utils.audio import AudioProcessor
from TTS.tts.configs.vits_config import VitsConfig
from TTS.tts.models.vits import Vits
from TTS.utils.text.tokenizer import TTSTokenizer
from TTS.tts.datasets import load_tts_samples
from trainer import Trainer, TrainerArgs

# Configuración del modelo VITS
config = VitsConfig(
    output_path='output',  # Ruta donde se guardarán los resultados del entrenamiento
    run_name='run',  # Nombre del experimento
    model='vits',  # Tipo de modelo
    epochs=1000,  # Número total de épocas para el entrenamiento
    batch_size=32,  # Tamaño del lote de datos para el entrenamiento
    lr=0.001,  # Tasa de aprendizaje para el optimizador
    optimizer='AdamW',  # Tipo de optimizador a utilizar
    save_step=10000,  # Número de pasos después del cual guardar el modelo
    save_checkpoints=True,  # Si se deben guardar puntos de control durante el entrenamiento
    audio={
        'sample_rate': 22050,  # Frecuencia de muestreo del audio (ejemplo: 22050 Hz)
        'n_fft': 2048,  # Tamaño de la ventana para la transformada de Fourier
        'hop_size': 512,  # Tamaño del salto entre ventanas
        'win_size': 2048,  # Tamaño de la ventana de análisis
        'preemphasis': 0.97,  # Coeficiente de preénfasis para mejorar la relación señal-ruido
        'min_db': -100,  # Umbral mínimo en decibelios para normalización
        'min_level_db': -100  # Umbral mínimo en decibelios para el audio
    }
)

# Inicialización del Procesador de Audio
ap = AudioProcessor(**config.audio.to_dict())

# Inicialización del Tokenizador
tokenizer, _ = TTSTokenizer.init_from_config(config)

# Función para preprocesar audio
def preprocess_audio(file_path, config):
    # Cargar audio
    y, sr = librosa.load(file_path, sr=config.audio['sample_rate'])

    # Normalización del volumen
    y = y / np.max(np.abs(y))

    # Transformaciones: Preénfasis
    y = np.append(y[0], y[1:] - config.audio['preemphasis'] * y[:-1])

    # Padding y Trimming
    target_length = config.audio['n_fft']
    if len(y) < target_length:
        y = np.pad(y, (0, target_length - len(y)), mode='constant')
    else:
        y = y[:target_length]

    return y

# Cargar y preprocesar datos
dataset_config = BaseDatasetConfig(
    formatter="ljspeech",
    meta_file_train="metadata.csv",
    path=os.path.join(config.output_path, "../LJSpeech-1.1/")
)

# Preprocesar un archivo de ejemplo
example_audio_path = os.path.join(config.output_path, "../LJSpeech-1.1/wavs/example.wav")
preprocessed_audio = preprocess_audio(example_audio_path, config)

# Cargar muestras de entrenamiento
train_samples, eval_samples = load_tts_samples(
    dataset_config,
    eval_split=True,
    eval_split_max_size=config.eval_split_max_size,
    eval_split_size=config.eval_split_size,
)

# Inicialización del Modelo VITS
model = Vits(config, ap, tokenizer)

# Inicialización del Entrenador
trainer = Trainer(
    TrainerArgs(), config, config.output_path, model=model, train_samples=train_samples, eval_samples=eval_samples
)

# Ejecución del Entrenamiento
trainer.fit()

# Evaluación del Modelo
trainer.evaluate()

# Exportación del Modelo
model.export_onnx(output_path='coqui_vits.onnx')


**Tiempo:**

  Duración del audio: Se maneja en la función preprocess_audio con el padding y trimming, asegurando que todos los audios sean de la longitud esperada.

**Frecuencia:**

  Frecuencia de muestreo: Especificada en config.audio['sample_rate'] para asegurar que todos los audios tengan esta frecuencia.

**Transformaciones:**

  Preénfasis: Se aplica en la función preprocess_audio para mejorar la relación señal-ruido.

**Longitud de los Segmentos de Audio:**

  Padding y Trimming: Ajusta la longitud del audio en la función preprocess_audio para que se ajuste a la longitud especificada.

**Ventanas y Superposición:**

  Tamaño de la ventana: Configurado con config.audio['n_fft'] y config.audio['hop_size'], afecta la transformada de Fourier.

**Especificaciones de Formato:**

  Formato del archivo: En este caso, se espera que el audio esté en formato WAV, pero puede ser ajustado según las necesidades del modelo.

**Normalización del Volumen:**

  Normalización: Se aplica en la función preprocess_audio para asegurar que el nivel de audio sea consistente.

**Ruido de Fondo y Limpieza:**

  Limpieza del audio: Aunque no se incluye un proceso de eliminación de ruido específico en este ejemplo, se puede agregar según los requisitos del modelo.

**Padding y Trimming:**

  Ajuste de longitud: Realizado en la función preprocess_audio para manejar audios demasiado cortos o largos.

**Especificaciones del Texto:**

  Preparación del texto: Asegúrate de que el texto esté en el formato necesario para el tokenizador, como se configura en TTSTokenizer.

4.- Ejemplo

In [1]:
import torch # Esta arquitectura ya la tenemos instalada
from vits import VITSModel
from utils import load_model, text_to_sequence

# Paso 1: Cargar el modelo preentrenado
model_path = "vits_model.pth"  # Ruta al modelo VITS preentrenado
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = load_model(model_path, device)

# Paso 2: Convertir texto a secuencia de caracteres/fonemas
text = "Hola, ¿cómo estás?" # Aqui podemos abrir los archivos de los textos de los audios
sequence = text_to_sequence(text)

# Paso 3: Convertir la secuencia en tensor
text_tensor = torch.LongTensor(sequence).unsqueeze(0).to(device)

# Paso 4: Generar la onda de audio usando el modelo VITS
with torch.no_grad():
    audio_output = model.inference(text_tensor)

# Paso 5: Guardar la salida como un archivo de audio
audio_output = audio_output.cpu().numpy()
save_wav(audio_output, "output.wav")

# Función para guardar el audio (por ejemplo en formato WAV)
import scipy.io.wavfile as wav

def save_wav(audio, filename, sr=22050):
    wav.write(filename, sr, (audio * 32767).astype('int16'))


ModuleNotFoundError: No module named 'vits'